In [25]:
import os
import cv2
import numpy as np
import pandas as pd

from insightface.app import FaceAnalysis

In [26]:
#configure face analysis
faceapp=FaceAnalysis(name='buffalo_sc',
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [27]:
#Extract Facial Features

In [32]:
person_info=[]
listdir=os.listdir(path='images')
for folder_name in listdir:
    role,name=folder_name.split('-')

    #path of each image in respective folder
    img_files=os.listdir(path=f'images/{folder_name}')
    for file in img_files:
        path=f'./images/{folder_name}/{file}'
        #step-1: read the images
        img_arr=cv2.imread(path)

        #step-2: get the info
        result=faceapp.get(img_arr,max_num=1)#return list
        if len(result)>0:
            #step-3: extract facial embedding
            res=result[0]
            embedding=res['embedding']
            #step-4 :save all info name,role,embedding in a list
            person_info.append([name,role,embedding])

In [33]:
dataframe=pd.DataFrame(person_info,columns=['Name','Role','Facial Features'])
dataframe #store in redis database

,Name,Role,Facial Features
0,Aadish Jain,student,"[-1.89651, -0.22361787, 2.3829994, 1.4845628, ..."
1,Abhinav Gangil,student,"[-0.6267061, 0.99143255, 1.8860981, 2.5386827,..."
2,Aditi Wekhande,student,"[0.61851066, -2.313673, 1.0664217, -0.24302286..."
3,Aditi Wekhande,student,"[1.3814317, -2.3694372, -0.35375035, 0.2203314..."
4,Aditya Kshitiz,student,"[0.5845894, 0.17067143, 0.4124964, 1.2453465, ..."
...,...,...,...
115,Vijit Balsori,student,"[-0.765196, 0.15019533, 0.5577423, 1.7181464, ..."
116,Vineet Verma,student,"[-0.9653477, 0.16919827, -0.56249446, -0.38980..."
117,Votte Sriyans,student,"[-2.150356, -1.7748274, 0.22421616, 0.1275861,..."
118,Votte Sriyans,student,"[-3.171238, -2.7374885, 0.13183016, 0.9242382,..."


In [34]:
img=cv2.imread('test_images/test1.jpg')
cv2.imshow("test",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
res_test=faceapp.get(img,max_num=0)
for rt in res_test:
    bbox=rt['bbox'].astype(int)
    score=int(rt['det_score']*100)
    embed_test=rt['embedding']

In [36]:
len(embed_test)

512

In [37]:
x_list=dataframe['Facial Features'].tolist()
x=np.asarray(x_list)

In [38]:
x.shape

(120, 512)

In [39]:
from sklearn.metrics import pairwise

In [40]:
y=embed_test.reshape(1,512) #1*512

cosine_similar=pairwise.cosine_similarity(x,y)

In [41]:
data_search=dataframe.copy()
data_search['cosine']=cosine_similar

In [42]:
data_search

,Name,Role,Facial Features,cosine
0,Aadish Jain,student,"[-1.89651, -0.22361787, 2.3829994, 1.4845628, ...",0.180937
1,Abhinav Gangil,student,"[-0.6267061, 0.99143255, 1.8860981, 2.5386827,...",0.244054
2,Aditi Wekhande,student,"[0.61851066, -2.313673, 1.0664217, -0.24302286...",0.069287
3,Aditi Wekhande,student,"[1.3814317, -2.3694372, -0.35375035, 0.2203314...",0.021748
4,Aditya Kshitiz,student,"[0.5845894, 0.17067143, 0.4124964, 1.2453465, ...",0.080312
...,...,...,...,...
115,Vijit Balsori,student,"[-0.765196, 0.15019533, 0.5577423, 1.7181464, ...",0.206987
116,Vineet Verma,student,"[-0.9653477, 0.16919827, -0.56249446, -0.38980...",0.050091
117,Votte Sriyans,student,"[-2.150356, -1.7748274, 0.22421616, 0.1275861,...",0.089664
118,Votte Sriyans,student,"[-3.171238, -2.7374885, 0.13183016, 0.9242382,...",0.000284


In [43]:
s_opt=0.5

In [45]:
datafilter=data_search.query(f'cosine>{s_opt}')
datafilter.reset_index(drop=True,inplace=True)

if len(datafilter)>0:
    argmax=datafilter['cosine'].argmax()
    name_cos,role_cos=datafilter.loc[argmax][['Name','Role']]
else:
    name_cos='Unknown'
    role_cos='Unknown'


In [46]:
print(name_cos,role_cos)

Vedant Upadhyay student


In [47]:
datafilter

,Name,Role,Facial Features,cosine
0,Vedant Upadhyay,student,"[-0.8973803, 1.2294897, 0.6510981, 1.2588146, ...",0.680172
1,Vedant Upadhyay,student,"[-0.8543678, 0.23283702, 1.2702205, 1.826755, ...",0.755717


In [48]:
def ml_search_algorithm(dataframe,feature_column,test_vector,name_role=['Name','Role'],thresh=0.5):
    #step-1: take the dataframe(collection of data)
    dataframe=dataframe.copy()
    #step-2: Index face embeding from the dataframe
    x_list=dataframe[feature_column].tolist()
    x=np.asarray(x_list)
    #step-3: Calculate cosine similarity
    similar=pairwise.cosine_similarity(x,test_vector.reshape(1,-1))
    similar_arr=np.array(similar).flatten()
    dataframe['cosine']=similar_arr
    #step-4: filter the data
    data_filter=dataframe.query(f'cosine>={thresh}')
    if len(data_filter)>0:
        #step-5: get the person name
        data_filter.reset_index(drop=True,inplace=True)
        argmax=data_filter['cosine'].argmax()
        person_name,person_role=data_filter.loc[argmax][name_role]
    else:
        person_name='Unknown'
        person_role='Unknown'

    return person_name,person_role
    

In [58]:
test_image=cv2.imread('test_images/test5.jpg')
cv2.imshow('test image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
#step-1: take the test image and apply to insight face
results=faceapp.get(test_image)
test_copy=test_image.copy()

#step-2: use for loop and extract each embedding and pass to ml_search_algorithm

for res in results:
    x1,y1,x2,y2=res['bbox'].astype(int)
    embeddings=res['embedding']
    person_name,person_role=ml_search_algorithm(dataframe,'Facial Features',test_vector=embeddings,name_role=['Name','Role'],thresh=0.5)
    cv2.rectangle(test_copy,(x1,y1),(x2,y2),(0,255,0))
    text_gen=person_name
    cv2.putText(test_copy,text_gen,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,0.35,(0,255,0),1)

cv2.imshow('test image',test_copy)
cv2.waitKey()
cv2.destroyAllWindows()
